<a href="https://colab.research.google.com/github/khushee-g/EEG/blob/main/With_error_measure_Exponentiated_Muse_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # Linear algebra
import pandas as pd
# ModelLing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics

In [ ]:
#Delta AF7-AF8 coherence - corresponds to COH.A.delta.a.FP1.b.FP2

from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = data['Delta_AF8'].values
channel_2 = data['Delta_AF7'].values

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF7DeltaAF8': Cxy})

coherence_AF8_AF7 = coherence_data['Coherence DeltaAF7DeltaAF8'].mean()
coherence_AF8_AF7 *=100
print(coherence_AF8_AF7)


13.365575864956632


<ipython-input-7-ed74ef2684ad>:5: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
#Delta AF8-TP9 coherence - corresponds to COH.A.delta.f.F4.h.T3



from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = data['Delta_AF8'].values
channel_2 = data['Delta_TP9'].values

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF8DeltaTP9': Cxy})

coherence_AF8_TP9 = coherence_data['Coherence DeltaAF8DeltaTP9'].mean()
coherence_AF8_TP9 *=100
print(coherence_AF8_TP9)

13.916182133081339


<ipython-input-6-c0e952f15780>:7: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
risk = 0
error_max = 0
error_min = 0
#coeff intercept + coeff channel val * (coherence*100)
#e^x


#COH.A.delta.a.FP1.b.FP2
AF7_AF8 = 0.0246*(coherence_AF8_AF7) - 1.7527
AF7_AF8 = np.exp(AF7_AF8)
AF7_AF8 -=1
risk += AF7_AF8

AF7_AF8_error_max = AF7_AF8 + (coherence_AF8_AF7*(0.0246+0.008)+(0.629-1.7527))
AF7_AF8_error_max = np.exp(AF7_AF8_error_max)
AF7_AF8_error_max -=1
error_max += AF7_AF8_error_max

AF7_AF8_error_min = AF7_AF8 - (coherence_AF8_AF7*(0.0246+0.008)+(0.629-1.7527))
AF7_AF8_error_min = np.exp(AF7_AF8_error_min)
AF7_AF8_error_min -=1
error_min += AF7_AF8_error_min



#COH.A.delta.f.F4.h.T3
AF8_TP9 = -0.0040*(coherence_AF8_TP9) + 0.1324
AF8_TP9 = np.exp(AF8_TP9)
AF8_TP9 -=1
risk += AF8_TP9

AF8_TP9_error_max = AF8_TP9 + (coherence_AF8_TP9*(-0.004+0.01)+(0.310+0.1324))
AF8_TP9_error_max = np.exp(AF8_TP9_error_max)
AF8_TP9_error_max -=1
error_max += AF8_TP9_error_max

AF8_TP9_error_min = AF8_TP9 - (coherence_AF8_TP9*(-0.004+0.01)+(0.310+0.1324))
AF8_TP9_error_min = np.exp(AF8_TP9_error_min)
AF8_TP9_error_min -=1
error_max += AF8_TP9_error_min



#AB.C.alpha.q.T6
TP10 = 0.0097*data['Delta_TP10'].mean() - 0.1531
TP10 = np.exp(TP10)
TP10 -=1
risk += TP10
TP10_error_max = TP10 + ((0.0097+0.007)*data['Delta_TP10'].mean() - (0.188+0.1531))
TP10_error_max = np.exp(TP10_error_max)
TP10_error_max -=1
error_max += TP10_error_max

TP10_error_min = TP10 - ((0.0097+0.007)*data['Delta_TP10'].mean() - (0.188+0.1531))
TP10_error_min = np.exp(TP10_error_min)
TP10_error_min -=1
error_max += TP10_error_min



#AB.B.theta.c.F7
AF7 = -0.0152 *data['Theta_AF7'].mean() + 0.2759
AF7 = np.exp(AF7)
AF7 -=1
risk += AF7

AF7_error_max = AF7 + ((-0.0152+0.382)*data['Delta_TP10'].mean() + (0.2759+0.024))
AF7_error_max = np.exp(AF7_error_max)
AF7_error_max -=1
error_max += AF7_error_max

AF7_error_min = AF7 - ((-0.0152+0.382)*data['Delta_TP10'].mean() + (0.2759+0.024))
AF7_error_min = np.exp(AF7_error_min)
AF7_error_min -=1
error_max += AF7_error_min


print("Error Min: " + str(error_min))
print("Error Max: " + str(error_max))
print(risk)

if risk > 0:
  prediction_val = (risk-1)*100
  print("You are " + str(prediction_val) + "% more likely to have behavioral disorder")

else:
  prediction_val = (1-(risk*-1))*100
  print("You are " + str(prediction_val) + " % less likely to have behavioral disorder")




Error Min: -0.06876745258049421
Error Max: 0.6535867950929523
-0.5020968357564883
You are 49.79031642435117 % less likely to have behavioral disorder
